<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Miami</font></h1>

## In this notebook, I will use the Foursquare API to explore neighborhoods in Miami to help new and prospective resients decide which neighborhoods in which to live. 

#### I will scrape the data table from Wikipedia. Then, I will use the explore function to get the most common venue categories in each neighborhood, and use this feature to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to complete this task. Finally, I'll use the Folium library to visualize the neighborhoods in Miami and their emerging clusters.

First, I download all the dependencies/libraries that I will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0           conda-forge
    geopy:         1.22.0-pyh9f0ad1d_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.22.0         | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All re

<a id='item1'></a>

## 1. Download and Explore Dataset

#### Miami has 25 official major neighborhoods. I was able to find a Wikipedia page (https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami) that lists each neighborhood with their respective longitudes and latitudes. I scraped the tables from the webpage and converted it to a pandas dataframe. Then, I cleaned the table, including splitting the longitude and latitude into separate columns.

Webscrape table from Wikipedia.

In [5]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami").text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
tab = str(soup.table)
display_html(tab,raw=True)

<title>List of neighborhoods in Miami - Wikipedia</title>


Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
Allapattah,,"54,289","4,401",,"25.815,-80.224"
Arts & Entertainment District,,"11,033","7,948",,"25.799,-80.190"
Brickell,Brickellite,"31,759","14,541",West Brickell,"25.758,-80.193"
Buena Vista,,"9,058","3,540",Buena Vista East Historic District and Design District,"25.813,-80.192"
Coconut Grove,Grovite,"20,076","3,091","Center Grove, Northeast Coconut Grove, Southwest Coconut Grove, and the West Grove (Black Grove)","25.712,-80.257"
Coral Way,,"35,062","4,496","Coral Gate, Golden Pines, Shenandoah, Historic Shenandoah, and Silver Bluff","25.750,-80.283"
Design District,,"3,573","3,623",,"25.813,-80.193"
Downtown,Downtowner,"71,000 (13,635 CBD only)","10,613","Brickell, Central Business District (CBD), Downtown Miami Historic District, Jewelry District, Lummus Park, Arts & Entertainment District, and Park West","25.774,-80.193"
Edgewater,,"15,005","6,675",,"25.802,-80.190"
Flagami,,"50,834","5,665","Alameda, Grapeland Heights, and Fairlawn","25.762,-80.316"


Convert HTML table to pandas dataframe.

In [6]:
dfs = pd.read_html(tab)
df=dfs[0]
df

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257
5,Coral Way,NaN,35062,4496,"Coral Gate, Golden Pines, Shenandoah, Historic...",25.750-80.283
6,Design District,NaN,3573,3623,NaN,25.813-80.193
7,Downtown,Downtowner,"71,000 (13,635 CBD only)",10613,"Brickell, Central Business District (CBD), Dow...",25.774-80.193
8,Edgewater,NaN,15005,6675,NaN,25.802-80.190
9,Flagami,NaN,50834,5665,"Alameda, Grapeland Heights, and Fairlawn",25.762-80.316


In [12]:
df.shape

(26, 6)

Drop unnecessary columns and split 'Coordinates' columns into seperate 'Longitude' and 'Latitude' columns. 

While checking the dataframe, I notice that Health District does not have coordinates. Upon further research, I learn that it is actually a subset of Downtown where there is a hub of hospitals, health centers and research facilities. Because this does not seem relevent to the problem of where to live (the Health District is more a hub of workplaces than of residence. Plus, if you choose Downtown, you will be near the Health District anyway), I will delete this row from the dataframe. 

By deleting 'NaN' rows, this will remove the Health District and the totals row at the bottom.

In [16]:
# drop unnecessary columns
df1 = df.drop(df.columns[[1,2,3,4]], axis=1)

#split 'Coordinates' column into 2 seperate 'Latitude' & 'Longitude' columns
df1[['Latitude','Longitude']] = df1.Coordinates.str.split("-",expand=True)
df1 = df1.drop('Coordinates',axis=1)

#drop rows with no Coordinates - removed Health District and totals row
df1 = df1.dropna()

df1.head()


,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,80.224
1,Arts & Entertainment District,25.799,80.190
2,Brickell,25.758,80.193
3,Buena Vista,25.813,80.192
4,Coconut Grove,25.712,80.257


In [14]:
#double-check that we have all neighborhoods - it should be 24 since we removed Health District
df1.shape

(24, 3)

Because I used the negative sign of the Longitude as the delimiter to split the coordinates column, it has been removed. Now I will add the negative sign back to all of the Longitude values. 

In [17]:
#add negative sign back onto longitudes
df1['Longitude'] = '-'+df1['Longitude'].astype(str)

df1.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


In [21]:
#change Longitude and Latitude to real numbers (necessary for map later)
df1.Latitude = df1.Latitude.astype(float)
df1.Longitude = df1.Longitude.astype(float)
df1.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


## 2. Exploring the Neighborhoods

#### In this section, I will first show a map of Miami and its neighborhoods. Then, using Foursquare, I will explore each neighborhood by pulling the top venues of each area and  determining what categories to which those venues belong.

First, use geopy library to get the latitude and longitude values of Miami.

In [18]:
address = 'Miami, FL'

geolocator = Nominatim(user_agent="mia_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Miami are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Miami are 25.7742658, -80.1936589.


Create a map of Miami with neighborhoods superimposed on top.

In [24]:
# create map of Manhattan using latitude and longitude values
map_miami = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_miami)  
    
map_miami

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'KFIOUWI352JNHBSYNIHDZM0VERNORQJOZMRMSQKIVAV4LRNY' 
CLIENT_SECRET = '03BBMHU3JH35BYSRXYMUX3DH4TVOQWNUNNTBVNJYIFICPDTB'
VERSION = '20200501' # (used 20180605 in class)

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KFIOUWI352JNHBSYNIHDZM0VERNORQJOZMRMSQKIVAV4LRNY
CLIENT_SECRET:03BBMHU3JH35BYSRXYMUX3DH4TVOQWNUNNTBVNJYIFICPDTB


This function will pull the venues data for a neighborhood using each neighborhood's latitude and longitude. It returns the name of the venue, its coordinates, and its category (to be used later for segmenting and clustering). 
I am limiting it to the top 50 venues in each neighborhood and radius 500. 

In [42]:
LIMIT = 50
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

print("Function defined")

Function defined


Now I run the above function on each neighborhood and create a new dataframe called **miami_venues**.

In [43]:

miami_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )



Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood


Check the size of the resulting dataframe. Being that there should be no more than 50 venues per neighborhood, it should not be larger than 1200 rows.

In [44]:
print(miami_venues.shape)
miami_venues.head()

(459, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,25.815,-80.224,Ross,25.815820,-80.221753,Department Store
1,Allapattah,25.815,-80.224,noor market,25.818165,-80.224197,Convenience Store
2,Arts & Entertainment District,25.799,-80.190,Bunnie Cakes,25.799544,-80.190953,Cupcake Shop
3,Arts & Entertainment District,25.799,-80.190,Bunbury Miami,25.798284,-80.191118,Wine Shop
4,Arts & Entertainment District,25.799,-80.190,Yodi's Threading Spa,25.800490,-80.189093,Spa


Check how many venues were returned for each neighborhood

In [45]:
miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,2,2,2,2,2,2
Arts & Entertainment District,21,21,21,21,21,21
Brickell,50,50,50,50,50,50
Buena Vista,27,27,27,27,27,27
Coconut Grove,4,4,4,4,4,4
Coral Way,11,11,11,11,11,11
Design District,26,26,26,26,26,26
Downtown,47,47,47,47,47,47
Edgewater,49,49,49,49,49,49


And finally, find out how many unique categories can be curated from all the returned venues.

In [46]:
print('There are {} uniques categories.'.format(len(miami_venues['Venue Category'].unique())))

There are 142 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

#### In this section, I will use one hot encoding to create a dataframe with dummy variable columns showing which categories of venues were present in each neighborhood. Then I will pull each neighborhood's top 3 most common category types and create a dataframe listing them. I will only use the top 3 venues because only half of the neighborhoods returned 10 or more venues and six of them (25% of the major neighborhoods) returned less than 5 venues.

In [47]:
# one hot encoding
miami_onehot = pd.get_dummies(miami_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
miami_onehot['Neighborhood'] = miami_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [miami_onehot.columns[-1]] + list(miami_onehot.columns[:-1])
miami_onehot = miami_onehot[fixed_columns]

miami_onehot.head()

,Yoga Studio,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beach,Big Box Store,Bistro,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Service,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Print Shop,Pub,Public Art,Record Shop,Rental Service,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
miami_onehot.shape

(459, 142)

Next, group rows by neighborhood and find the mean of the frequency of occurrence of each category.

In [49]:
miami_grouped = miami_onehot.groupby('Neighborhood').mean().reset_index()
miami_grouped

,Neighborhood,Yoga Studio,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beach,Big Box Store,Bistro,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Service,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Print Shop,Pub,Public Art,Record Shop,Rental Service,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint
0,Allapattah,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Arts & Entertainment District,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.095238,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [50]:
miami_grouped.shape

(24, 142)

Print each neighborhood along with the top 3 most common venues.

In [63]:
num_top_venues = 3

for hood in miami_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = miami_grouped[miami_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allapattah----
               venue  freq
0   Department Store   0.5
1  Convenience Store   0.5
2        Yoga Studio   0.0


----Arts & Entertainment District----
            venue  freq
0  Ice Cream Shop  0.10
1      Restaurant  0.10
2             Spa  0.05


----Brickell----
                 venue  freq
0                Hotel  0.10
1   Italian Restaurant  0.08
2  Japanese Restaurant  0.06


----Buena Vista----
                venue  freq
0  Italian Restaurant  0.07
1            Boutique  0.07
2       Jewelry Store  0.07


----Coconut Grove----
                 venue  freq
0  American Restaurant  0.25
1                 Park  0.25
2        Boat or Ferry  0.25


----Coral Way----
          venue  freq
0  Liquor Store  0.18
1  Burger Joint  0.09
2   Golf Course  0.09


----Design District----
                venue  freq
0  Italian Restaurant  0.08
1       Jewelry Store  0.08
2                Café  0.08


----Downtown----
                 venue  freq
0                Hotel  0.09
1    

And put that into a pandas dataframe by first using a function to sort the venues in descending order then creating a new datafram that displays the top 3 venues for each neighborhood.

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = miami_grouped['Neighborhood']

for ind in np.arange(miami_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(miami_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,Convenience Store,Department Store,Wings Joint
1,Arts & Entertainment District,Ice Cream Shop,Restaurant,Yoga Studio
2,Brickell,Hotel,Italian Restaurant,Japanese Restaurant
3,Buena Vista,Café,Jewelry Store,Boutique
4,Coconut Grove,Boat or Ferry,Park,Photography Studio


## 4. Cluster Neighborhoods

#### In this section I will use k-Means clustering to cluster the neighborhoods into 5 groups based on the venue categories. Then, we will see these clusters on a map of Miami.

Run *k*-means to cluster the neighborhoods into 5 clusters.

In [89]:
# set number of clusters
kclusters = 5

miami_grouped_clustering = miami_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(miami_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 4, 1, 1, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 3 venues for each neighborhood.

In [86]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

miami_merged = df1

# merge sorted venue df with df1 (original dataframe) to add latitude/longitude for each neighborhood
miami_merged = miami_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

miami_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,25.815,-80.224,0,Convenience Store,Department Store,Wings Joint
1,Arts & Entertainment District,25.799,-80.190,1,Ice Cream Shop,Restaurant,Yoga Studio
2,Brickell,25.758,-80.193,1,Hotel,Italian Restaurant,Japanese Restaurant
3,Buena Vista,25.813,-80.192,1,Café,Jewelry Store,Boutique
4,Coconut Grove,25.712,-80.257,4,Boat or Ferry,Park,Photography Studio


Finally, let's visualize the resulting clusters

In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miami_merged['Latitude'], miami_merged['Longitude'], miami_merged['Neighborhood'], miami_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### We can see that with only 5 clusters, there are a disproportionate number of neighborhoods in Cluster 2 (purple markers, 'Cluster 1'). I want to break up that large cluster a little more, so I will do another clustering model with 8 groups instead. 

Change the number of clusters to 8.

In [108]:
# set number of clusters
kclusters = 8

miami_grouped_clustering = miami_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(miami_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 1, 1, 1, 7, 1, 1, 1, 1, 6], dtype=int32)

Create a new dataframe showing the neighborhoods and all the relevant info again.

In [109]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

miami_merged = df1

# merge sorted venue df with df1 (original dataframe) to add latitude/longitude for each neighborhood
miami_merged = miami_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

miami_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,25.815,-80.224,5,Convenience Store,Department Store,Wings Joint
1,Arts & Entertainment District,25.799,-80.190,1,Ice Cream Shop,Restaurant,Yoga Studio
2,Brickell,25.758,-80.193,1,Hotel,Italian Restaurant,Japanese Restaurant
3,Buena Vista,25.813,-80.192,1,Café,Jewelry Store,Boutique
4,Coconut Grove,25.712,-80.257,7,Boat or Ferry,Park,Photography Studio


And map it all again to see if there was a significant improvement with more clusters. 

In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miami_merged['Latitude'], miami_merged['Longitude'], miami_merged['Neighborhood'], miami_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 2 (purple markers, 'Cluster 1') is still a large cluster, but it does have fewer neighborhoods than before. 

## 5. Examine Clusters

#### In this section, I will examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I will assign a name to each cluster.

#### Cluster 1

In [111]:
miami_merged.loc[miami_merged['Cluster Labels'] == 0]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
14,Little Havana,25.773,-80.215,0,Latin American Restaurant,Chinese Restaurant,Pharmacy


#### Cluster 2

In [112]:
miami_merged.loc[miami_merged['Cluster Labels'] == 1]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Arts & Entertainment District,25.799,-80.190,1,Ice Cream Shop,Restaurant,Yoga Studio
2,Brickell,25.758,-80.193,1,Hotel,Italian Restaurant,Japanese Restaurant
3,Buena Vista,25.813,-80.192,1,Café,Jewelry Store,Boutique
5,Coral Way,25.750,-80.283,1,Liquor Store,Pharmacy,Dive Bar
6,Design District,25.813,-80.193,1,Café,Jewelry Store,Boutique
7,Downtown,25.774,-80.193,1,Hotel,Pharmacy,Italian Restaurant
8,Edgewater,25.802,-80.190,1,Restaurant,Sandwich Place,Breakfast Spot
10,Grapeland Heights,25.792,-80.258,1,Hotel,Gym,Restaurant
13,Little Haiti,25.824,-80.191,1,Yoga Studio,Liquor Store,Pub
16,Midtown,25.807,-80.193,1,Coffee Shop,Peruvian Restaurant,Indian Restaurant


#### Cluster 3

In [113]:
miami_merged.loc[miami_merged['Cluster Labels'] == 2]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
17,Overtown,25.787,-80.201,2,Wings Joint,Southern / Soul Food Restaurant,Athletics & Sports


#### Cluster 4

In [114]:
miami_merged.loc[miami_merged['Cluster Labels'] == 3]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
12,Liberty City,25.832,-80.225,3,Southern / Soul Food Restaurant,Bar,Wings Joint


#### Cluster 5

In [115]:
miami_merged.loc[miami_merged['Cluster Labels'] == 4]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
22,Virginia Key,25.736,-80.155,4,Beach,Wings Joint,Food


#### Cluster 6

In [116]:
miami_merged.loc[miami_merged['Cluster Labels'] == 5] 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,25.815,-80.224,5,Convenience Store,Department Store,Wings Joint


#### Cluster 7

In [117]:
miami_merged.loc[miami_merged['Cluster Labels'] == 6] 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
9,Flagami,25.762,-80.316,6,Seafood Restaurant,Fast Food Restaurant,Bakery
15,Lummus Park,25.777,-80.201,6,Seafood Restaurant,Fish Market,Soccer Field


#### Cluster 8

In [118]:
miami_merged.loc[miami_merged['Cluster Labels'] == 7] 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,Coconut Grove,25.712,-80.257,7,Boat or Ferry,Park,Photography Studio
21,Venetian Islands,25.791,-80.161,7,Boat or Ferry,Lounge,Park


#### Based on the information above, the clusters can be named and described as below. I noticed that clusters 3 and 4 were very similar, so I've merged them myself. Cluster 2 has too many items, but there was much less information available through Foursquare than expected. I imagine if more venues were available for the various neighborhoods, the clusters would be more precise and Cluster 2 would be broken down more. 

|Cluster | Name | Description|
|:------:|:----:|:-----------|
|1     | "Latin Love" | Little Havana - Latin Restaurants
|2     | "Tourist Town" | City Center - Coffee, Hotels and Jewelry Stores
|3 & 4 | "Southern Comfort" | Soul Food and Wings Restaurants
|5     | "Fun in the Sun" | Beaches
|6     | "Family First" | Convenience and Department Stores (map shows Elem & Mid Schools)
|7     | "Happy as a Clam" | Seafood Restaurants
|8     | "Let's Sail Away" | Boat or Ferry and Parks